In [8]:
import requests
import pandas as pd
from selenium import webdriver

In [21]:
import sys
sys.path.append('../')
from common import get_train, get_test, get_parking

In [24]:
def swap_streets(input_df):
    replace = {'Geary Street,Jones Street,Geary Street':'Geary Street,Jones Street,Leavenworth Street',
     'Jones Street,Sutter Street,Jones Street': 'Jones Street,Sutter Street,Post Street',
     'Kearny Street,Kearny Street,Columbus Avenue': 'Kearny Street,Jackson Street,Columbus Avenue',
     'Kearny Street,Kearny Street,Post Street': 'Kearny Street,Maiden Lane,Post Street',
     'Kearny Street,Kearny Street,Sutter Street': 'Kearny Street,Post Street,Sutter Street',
     'Larkin Street,Larkin Street,Golden Gate Avenue': 'Larkin Street,Fulton Street,Golden Gate Avenue',
     "Larkin Street,O'Farrell Street,Larkin Street": "Larkin Street,O'Farrell Street,Myrtle Street",
     'Larkin Street,Sutter Street,Larkin Street': 'Larkin Street,Sutter Street,Bush Street',
     'Mason Street,Sutter Street,Mason Street': 'Mason Street,Sutter Street,Bush Street',
     'Mission Street,11th Street,Mission Street': 'Mission Street,11th Street,12th Street',
     'Montgomery Street,Jackson Street,Montgomery Street': 'Montgomery Street,Jackson Street,Washington Street',
     'Montgomery Street,Montgomery Street,Jackson Street': 'Montgomery Street,Pacific Avenue,Jackson Street',
     'Montgomery Street,Washington Street,Montgomery Street': 'Montgomery Street,Washington Street,Clay Street',
     'Post Street,Kearny Street,Post Street': 'Post Street,Kearny Street,Montgomery Street',
     'Stockton Street,Stockton Street,Jackson Street': 'Stockton Street,Washington Street,Jackson Street',
     'Stockton Street,Stockton Tunnel,Stockton Street': 'Stockton Street,Stockton Tunnel,Clay Street',
     'Sutter Street,Powell Street,Sutter Street': 'Sutter Street,Powell Street,Mason Street',
     'Van Ness Avenue,Golden Gate Avenue,Van Ness Avenue': 'Van Ness Avenue,Golden Gate Avenue,Turk Street',
     'Van Ness Avenue,Hayes Street,Van Ness Avenue': 'Van Ness Avenue,Hayes Street,Grove Street',
     'Van Ness Avenue,McAllister Street,Van Ness Avenue': 'Van Ness Avenue,McAllister Street,Golden Gate Avenue',
     'Van Ness Avenue,Van Ness Avenue,Bush Street': 'Van Ness Avenue,Sutter Avenue,Bush Street',
     'Van Ness Avenue,Van Ness Avenue,Eddy Street': 'Van Ness Avenue,Turk Street,Eddy Street',
     'Van Ness Avenue,Van Ness Avenue,Fell Street': 'Van Ness Avenue,Oak Street,Fell Street',
     'Van Ness Avenue,Van Ness Avenue,Golden Gate Avenue': 'Van Ness Avenue,McAllister Street,Golden Gate Avenue',
     'Van Ness Avenue,Van Ness Avenue,Grove Street': 'Van Ness Avenue,Hayes Street,Grove Street',
     'Van Ness Avenue,Van Ness Avenue,Turk Street': 'Van Ness Avenue,Golden Gate Avenue,Turk Street'}

    replace_fields = [k.split(',') + v.split(',') for k,v in replace.items()]
    replace_df = pd.DataFrame(replace_fields)
    replace_df.columns = ['Street', 'From', 'To', 'Clean_Street','Clean_From','Clean_To']

    proxy = input_df.merge(replace_df, how='left', on=['Street','From','To'])
    proxy.loc[proxy['Clean_Street'].isna(), 'Clean_Street'] = proxy.loc[proxy['Clean_Street'].isna(), 'Street']
    proxy.loc[proxy['Clean_From'].isna(), 'Clean_From'] = proxy.loc[proxy['Clean_From'].isna(), 'From']
    proxy.loc[proxy['Clean_To'].isna(), 'Clean_To'] = proxy.loc[proxy['Clean_To'].isna(), 'To']
    return proxy

In [25]:
train_df = get_train()
test_df = get_test()

train_df = swap_streets(train_df)
test_df = swap_streets(test_df)

Time it took: 0.17236900329589844
Time it took: 0.08989810943603516


In [28]:
concat = pd.concat([train_df, test_df])

In [34]:
for st, from_st, to_st, sz in concat.groupby(['Clean_Street','Clean_From','Clean_To']).size().reset_index().values:
    print(st,'|', from_st,'|', to_st)

23rd Street | Bartlett Street | Valencia Street
23rd Street | Mission Street | Bartlett Street
23rd Street | Mission Street | Capp Street
23rd Street | South Van Ness Avenue | Capp Street
Battery Street | California Street | Pine Street
Battery Street | Halleck Street | California Street
Battery Street | Washington Street | Merchant Street
Bryant Street | Mariposa Street | 17th Street
Bush Street | Parking Entrance Lane | Chatham Place
Bush Street | Taylor Street | Mason Street
Columbus Avenue | Kearny Street | Pacific Avenue
Geary Street | Jones Street | Leavenworth Street
Geary Street | Larkin Street | Polk Street
Geary Street | Leavenworth Street | Hyde Street
Grove Street | Dr Carlton P Goodlett Place | Larkin Street
Grove Street | Dr Carlton P Goodlett Place | Van Ness Avenue
Hyde Street | Golden Gate Avenue | McAllister Street
Jackson Street | Polk Street | Van Ness Avenue
Jessie Street | New Montgomery Street | Annie Street
Jones Street | Antonio Street | Ellis Street
Jones Stre

In [9]:
df = pd.read_feather('../ref_data/gmaps_df_tt.feather')

In [20]:
for q, r, t in df[['Search_Addr','street_no','street_name']].values:
    print(q,'|', r,'|', t)

Mission Street and 25th Street, San Francisco | 2900 | Mission Street
Polk Street and Ellis Street, San Francisco | 801 | Polk Street
Van Ness Avenue and Geary Boulevard, San Francisco | 1060 | Geary Boulevard
Van Ness Avenue and Bush Street, San Francisco | 1400 | Van Ness Avenue
Van Ness Avenue and Daniel Burnham Court, San Francisco | 1207-1227 | Van Ness Avenue
Larkin Street and Cedar Street, San Francisco | 934 | Larkin Street
Pine Street and Larkin Street, San Francisco | 1300-1308 | Larkin Street
Pine Street and Polk Street, San Francisco | 1401 | Polk Street
Larkin Street and Myrtle Street, San Francisco | 820 | Larkin Street
Van Ness Avenue and Sutter Avenue, San Francisco | 1300 | Sutter Street
Larkin Street and Sutter Street, San Francisco | 1098 | Sutter Street
Larkin Street and McAllister Street, San Francisco | 270 | McAllister Street
23rd Street and Mission Street, San Francisco | 2701 | Mission Street
23rd Street and Bartlett Street, San Francisco | 201-205 | Bartlett S

Post Street and Meacham Place, San Francisco | 922 | Post Street
Polk Street and Elm Street, San Francisco | 505 | Polk Street
Geary Street and Polk Street, San Francisco | 1000 | Polk Street
Taylor Street and Bush Street, San Francisco | 800-826 | Taylor Street
Grove Street and Van Ness Avenue, San Francisco | 301 | Van Ness Avenue
Larkin Street and Golden Gate Avenue, San Francisco | 401 | Larkin Street
Geary Street and Hyde Street, San Francisco | 800 | Geary Street
Polk Street and Dr Tom Waddell Place, San Francisco | 101 | Polk Street
Hyde Street and McAllister Street, San Francisco | 201-223 | McAllister Street
Polk Street and Eddy Street, San Francisco | 700 | Polk Street
Grove Street and Larkin Street, San Francisco | 66 | Grove Street
Taylor Street and Eddy Street, San Francisco | 193 | Taylor Street
Taylor Street and Geary Street, San Francisco | 511 | Taylor Street
Mason Street and Bush Street, San Francisco | 798 | Bush Street
Sutter Street and Taylor Street, San Francisco 

In [10]:
df.head()

,Street1,Street2,Search_Addr,address_tags,lat,lng,neighborhood,related_addr,street_name,street_no,zipcode
0,Mission Street,25th Street,"Mission Street and 25th Street, San Francisco",None,37.750648,-122.418316,Mission District,"[['2900 Mission St, San Francisco, CA 94110, U...",Mission Street,2900,94110
1,Polk Street,Ellis Street,"Polk Street and Ellis Street, San Francisco","bus_station,establishment,point_of_interest,tr...",37.784040,-122.419381,Tenderloin,"[['801 Polk St, San Francisco, CA 94109, USA',...",Polk Street,801,94109
2,Van Ness Avenue,Geary Boulevard,"Van Ness Avenue and Geary Boulevard, San Franc...","bus_station,establishment,point_of_interest,tr...",37.785696,-122.421327,Tenderloin,"[['1060 Geary Blvd, San Francisco, CA 94109, U...",Geary Boulevard,1060,94109
3,Van Ness Avenue,Bush Street,"Van Ness Avenue and Bush Street, San Francisco","establishment,general_contractor,local_governm...",37.788499,-122.421892,Nob Hill,"[['1400 Van Ness Ave, San Francisco, CA 94109,...",Van Ness Avenue,1400,94109
4,Van Ness Avenue,Daniel Burnham Court,"Van Ness Avenue and Daniel Burnham Court, San ...","bus_station,establishment,point_of_interest,tr...",37.787084,-122.421757,Western Addition,"[['1207-1227 Van Ness Ave, San Francisco, CA 9...",Van Ness Avenue,1207-1227,94109


In [14]:


with open('/Users/timlee/Dropbox/keys/google_api_key.txt','r') as f:
    gmap_api_key = f.read()
gmaps = googlemaps.Client( key = gmap_api_key)
gmaps.directions("Mission street and 25th street, San Francisco",
                 "Mission street and 26th street, San Francisco",
                 mode="walking")

[{'bounds': {'northeast': {'lat': 37.7506482, 'lng': -122.4181659},
   'southwest': {'lat': 37.749044, 'lng': -122.4183159}},
  'copyrights': 'Map data ©2018 Google',
  'legs': [{'distance': {'text': '0.1 mi', 'value': 179},
    'duration': {'text': '2 mins', 'value': 125},
    'end_address': 'Mission St & 26th St, San Francisco, CA 94110, USA',
    'end_location': {'lat': 37.749044, 'lng': -122.4181659},
    'start_address': 'Mission St & 25th St, San Francisco, CA 94110, USA',
    'start_location': {'lat': 37.7506482, 'lng': -122.4183159},
    'steps': [{'distance': {'text': '0.1 mi', 'value': 179},
      'duration': {'text': '2 mins', 'value': 125},
      'end_location': {'lat': 37.749044, 'lng': -122.4181659},
      'html_instructions': 'Head <b>south</b> on <b>Mission St</b> toward <b>26th St</b>',
      'polyline': {'points': 'qdleFnydjVz@Ez@EhAGd@CT?bAE'},
      'start_location': {'lat': 37.7506482, 'lng': -122.4183159},
      'travel_mode': 'WALKING'}],
    'traffic_speed_entry